In [2]:
import os
import numpy as np
import torch
from PIL import Image
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from engine import train_one_epoch, evaluate
import utils
import transforms as T
import h5py

In [3]:
# Created the Class for the custom dataset
class CustomDataset(torch.utils.data.Dataset):
    
    def __init__(self, root_img, root_mask, root_npy_labels, root_npy_bboxes, transforms = None):
        
        """
        Inputs:
        root_img: The path to the root directory where the image .h5 files are stored
        root_mask: The path to the root directory where the mask .h5 files are stored
        root_npy_labels: The path to the .npy dataset for labels
        root_npy_bboxes: The path to the .npy dataset for the ground truth bounding boxes
        transforms: Apply a Pytorch transform to each instance of the image
        
        """
        self.root_img = root_img
        self.root_mask = root_mask
        self.root_npy_labels = root_npy_labels
        self.root_npy_bboxes = root_npy_bboxes
        self.transforms = transforms 
        
        self.imgs = h5py.File(self.root_img, 'r')
        self.mask = h5py.File(self.root_mask, 'r')
        self.labels = np.load(self.root_npy_labels, allow_pickle = True)
        self.bboxes = np.load(self.root_npy_bboxes, allow_pickle = True)
        
    # To support indexing when an object of the CustomDataset Class is created
    def __getitem__(self, index):
        
        # Convert the Masks and the input image into an array
        image = np.array(self.imgs['data']).astype('int32')
        masks = np.array(self.mask['data']).astype('int32')
        
        # Convert the Mask, image, bounding boxes and labels to a Pytorch Tensor
        image = torch.as_tensor(image[index])
        masks = torch.as_tensor(masks[index])
        bounding_boxes = torch.as_tensor(self.bboxes[index])
        labels = torch.as_tensor(self.labels[index])
        
        batch = {} 
        batch["boxes"] = bounding_boxes
        batch["masks"] = masks
        batch["labels"] = labels
        
        if self.transforms is not None:
            image, batch = self.transforms(image,batch)
        return image, batch

    def __len__(self):
        return len(self.imgs)

In [4]:
class PennFudanDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)

        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [3]:
root1 = 'mycocodata_img_comp_zlib.h5'
root2 = 'mycocodata_mask_comp_zlib.h5'
root3_npy = 'mycocodata_labels_comp_zlib.npy'
root4_npy = 'mycocodata_bboxes_comp_zlib.npy'

dataset = CustomDataset(root1, root2, root3_npy, root4_npy)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=utils.collate_fn)

(<PIL.Image.Image image mode=RGB size=559x536 at 0x7FA7AB3AEA90>,
 {'area': tensor([35358., 36225.]), 'boxes': tensor([[159., 181., 301., 430.],
          [419., 170., 534., 485.]]), 'image_id': tensor([0]), 'iscrowd': tensor([0, 0]), 'labels': tensor([1, 1]), 'masks': tensor([[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]],
  
          [[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8)})

In [ ]:
dataset = PennFudanDataset('/content/drive/My Drive/PennFudanPed', get_transform(train=True))
dataset_test = PennFudanDataset('/content/drive/My Drive/PennFudanPed', get_transform(train=False))
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])
data_loader = torch.utils.data.DataLoader(dataset, batch_size = 2, shuffle = True, num_workers = 4, collate_fn = utils.collate_fn)
data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size = 1, shuffle = False, num_workers = 4, collate_fn = utils.collate_fn)

In [4]:
# create device variable
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# set num_classes
num_classes = 4

# get the model using our helper function
model = get_model_instance_segmentation(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr = 0.005, momentum = 0.9, weight_decay = 0.0005)
    
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)

# let's train it for 10 epochs
num_epochs = 10

In [5]:
# required functions
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
# train model
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq = 10)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)